In [1]:
import gym
from gym.spaces import Box, Discrete, Tuple, MultiDiscrete
import logging
import random
from PIL import Image
from pypylon import pylon
from ray.rllib.env import MultiAgentEnv
import ray
from ctypes import *
import os
from ray import tune
from ray.tune import function
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import PPOTrainer, PPOTFPolicy, PPOTorchPolicy
import ray.rllib.agents.ppo as ppo
import numpy as np
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from ray.tune.logger import pretty_print
import numpy as np
import cv2
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym

from gym import error
from gym.utils import closer
from gym import spaces
import time
from matplotlib import cm
import random
import torch
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
from torchvision import models, transforms
from torch import nn
import imquality.brisque as brisque
import csv

import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
logger = logging.getLogger(__name__)

In [3]:
class Camera_Env(gym.Env):
    """The main OpenAI Gym class. It encapsulates an environment with
    arbitrary behind-the-scenes dynamics. An environment can be
    partially or fully observed.
    The main API methods that users of this class need to know are:
        step
        reset
        render
        close
        seed
    And set the following attributes:
        action_space: The Space object corresponding to valid actions
        observation_space: The Space object corresponding to valid observations
        reward_range: A tuple corresponding to the min and max possible rewards
    Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
    The methods are accessed publicly as "step", "reset", etc...
    
    """
    metadata = {'render.modes': ['rgb_array']}
    
    def __init__(self, 
                 camera,
                 file_path = r'D:\expos_time'):
        
        
        # Create an instant camera object with the camera device found first.
        #self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        #self.camera.Open()
        self.camera = camera
        self.camera.AcquisitionFrameRateAbs.Value = 10
        self.camera.GainRaw.Value = 36
        self.camera.ExposureTimeRaw.Value = 20000
        self.camera.AcquisitionMode.SetValue('Continuous')
        self.camera.Width.Value = 417
        self.camera.Height.Value = 404
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
#         # corning lib
#         self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
#         #Check if Maxim driver dll is loaded
#         eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
#         print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        
        self.range_ex_time = np.concatenate((np.arange(1e3, 45e3, 1000), 
                                             np.arange(50e3, 500e3, 5000), 
                                             np.arange(600e3, 900e3, 50000), 
                                             np.arange(100e4, 500e4, 500000))).astype(np.int64)
        
        number_of_actions = len(self.range_ex_time)
        print(number_of_actions)
        
#         self.resnet152 = models.resnet152(pretrained=True)
#         modules=list(self.resnet152.children())[:-1]
#         self.resnet152=nn.Sequential(*modules)
#         for p in self.resnet152.parameters():
#             p.requires_grad = False
        
        self.action_space = spaces.Discrete(number_of_actions)
        #self.observation_space = spaces.Box(0, np.inf, (21, ))
        self.observation_space = spaces.Tuple((spaces.Box(0, 2, (2048, )),
                                               spaces.Box(0, np.inf, (10, )), 
                                               spaces.Box(0, np.inf, (11, ))
                                              ))
        
    def basler(self, action):  
        ExposureTimeRaw = int(self.range_ex_time[action])
        print('ExposureTimeRaw:', ExposureTimeRaw) 
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        time.sleep(3) 
        
    def grab_image(self):
        # https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                #print(img[0])
                #print('shape:', img.shape)
                #plt.imshow(img)
                break
        self.camera.StopGrabbing()
        return image
    
    def hist(self, image):
        hist, bin_edges =np.histogram(np.array(image).ravel())
        #image_inf = np.concatenate((hist, bin_edges))
        
        max_value_bin = bin_edges[np.argmax(hist)]
        if max_value_bin >= 50 and max_value_bin <= 150:
            image_asses = 1
        elif (max_value_bin > 25 and max_value_bin < 50) or (max_value_bin > 150 and max_value_bin < 175):
            image_asses = - 0.01*round(brisque.score(image), 2)
        else:
            image_asses = -1
        return hist, bin_edges, image_asses
        
    def reset(self):
        self.camera.ExposureTimeRaw.Value = 20000
        time.sleep(3) 
        
#         image = self.grab_image()
#         hist, bins, _ = self.hist(image)
        
#         obs_feat = self.take_features(image)
        
#         obs = (obs_feat, hist, bins)
        
#         return obs
    
    def step(self, action):
        pass
#         self.basler(action)
#         image = self.grab_image()
#         hist, bins, image_asses = self.hist(image)
# #         print('Brisque:', image_asses)
        
#         done = True
#         if image_asses == -1:
#             done = False
                 
# #         filepath = r'D:\expos_time'
# #         im = Image.fromarray(image)
# #         filename = filepath + "\img_%s_%s.png" % (str(action), str(image_asses))
# #         im.save(filename)
        
#         obs_feat = self.take_features(image)
        
#         obs = (obs_feat, hist, bins)

#         return obs, image_asses, done, {}

In [4]:
class Lens_Env(gym.Env):
    def __init__(self, camera,
                        Height = 404, 
                         Width = 417):
        
        self.camera = camera
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        self.Width = Width
        self.Height = Height
        
        self.action_space = MultiDiscrete([69, 99])
        # image
        self.observation_space = Box(0, 1, (2048, ))#(len(np.zeros((404, 417, 3)).flatten()),))
#         self.observation_space = Box(low=0, 
#                                             high=255, 
#                                             shape=(self.Height, self.Width, 3),
#                                             dtype = np.uint8)
        self.camera_env = Camera_Env(self.camera)
        print('---------------lens env init-------------------')
    
    def lens_movement(self, action):
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(3)
        
    def reset(self):
        self.lens_movement(46.0)
        
    def step(self, action):
        self.lens_movement(action)
        img = self.camera_env.grab_image()
        return img, -1, False, {}

In [5]:
class BaslerEnv(MultiAgentEnv):
    def __init__(self, 
                       Width = 417,
                       Height = 404,
                       threshold = 30):
        
        self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        self.camera.Open()

        self.basler_env = Camera_Env(self.camera)
        self.corning_env = Lens_Env(self.camera)
        
        # yolo
        self.model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', 
                                         force_reload=True, 
                                         pretrained=True)
        
        self.resnet152 = models.resnet152(pretrained=True)
        modules=list(self.resnet152.children())[:-1]
        self.resnet152=nn.Sequential(*modules)
        for p in self.resnet152.parameters():
            p.requires_grad = False
        
        print(True)
        self.histogram_done = False
        
        
        
    def take_features(self, img):
        preprocess = transforms.Compose([
                            transforms.Resize(256),
                            transforms.CenterCrop(224),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
        
        image = Image.fromarray(np.uint8(img)).convert('RGB')
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
        
        # Run through the convolutional layers and resize the output.
        features_output = self.resnet152(input_batch)
        classifier_input = features_output.view(1, -1)
        
        return classifier_input.cpu().detach().numpy()[0]
    
    def reset(self):

        self.basler_env.reset()
        self.corning_env.reset()
        
        self.steps_remaining_at_level = None
        self.num_high_level_steps = 0
        
        self.low_level_agent_id = "low_level_{}".format(self.num_high_level_steps)
        
        image = self.basler_env.grab_image()
        hist, bins, image_asses = self.basler_env.hist(image)
        obs_feat = self.take_features(image)
        obs = (obs_feat, hist, bins)
        self.histogram_done = False
        
        return {"high_level_agent": obs,}
    
    def find_class(self, results, obj = 'car'):
        len_of_class = len(results.pred)
        pred_class = [int(results.pred[0][i].numpy()[5]) for i in range(len(results.pred[0]))]
        #list_of_yolo_classes = list(zip(results.names, range(0, len(results.names))))
        pred_names = [results.names[i] for i in pred_class]
        if obj in pred_names:
            index = pred_names.index(obj)
            return results.pred[0][index].numpy()
        else:
            return np.array([])
        
    def step(self, action_dict):
        if "high_level_agent" in action_dict:
            return self._high_level_step(action_dict["high_level_agent"])
        else:
            return self._low_level_step(list(action_dict.values())[0])
    
    
    def _high_level_step(self, action):
        self.histogram_done = False
        self.action_main = action
        
        logger.debug("High level agent action".format(action))
        
        #self.obs_camera, reward_camera, done_c, _ = self.basler_env.step(action)
        self.basler_env.basler(action)
        #self.corning_env.lens_movement(46.0)
        self.obs_camera = self.basler_env.grab_image()
        
        self.steps_remaining_at_level = 10
        self.num_high_level_steps += 1
        self.low_level_agent_id = "low_level_{}".format(self.num_high_level_steps)
        
        feature_obs = self.take_features(self.obs_camera)
        obs = {self.low_level_agent_id: feature_obs}
        rew = {self.low_level_agent_id: 0}
        
        done = {"__all__": False}
        
        return obs, rew, done, {}
    
    def _low_level_step(self, action):
        
        action_lens_coarse = 0
        action_lens_fine = action[1]
        if action[0] < 24:
            action_lens_coarse = 24
        else:
            action_lens_coarse = action[0]
        action_lens = float("%s.%s"% (str(action_lens_coarse), str(action_lens_fine)))
        
        self.corning_env.lens_movement(action_lens)
        
        image = self.basler_env.grab_image()
        hist, bins, image_asses = self.basler_env.hist(image)
        obs_feat = self.take_features(image)
        #obs = (obs_feat, hist, bins)
        
        obs = {self.low_level_agent_id: obs_feat}
        if self.histogram_done == False:
            if image_asses != -1:
                self.histogram_done = True
        
        logger.debug("Low level agent step {}".format(action_lens))
        done = {"__all__": False}
        flag = False
        
        if self.histogram_done == True:
            self.steps_remaining_at_level -= 1
            
            prediction_yolo = self.model_yolo(image)
            final_result_yolo = self.find_class(prediction_yolo)
            
            if final_result_yolo.size == 0:
                rew = {self.low_level_agent_id: -1}
                if self.steps_remaining_at_level == 0:
                    done[self.low_level_agent_id] = True
                    rew["high_level_agent"] = image_asses
                    obs["high_level_agent"] = (obs_feat, hist, bins)
            
            else:
                flag = True
                image_ = Image.fromarray(image)
                image_ = image_.crop((final_result_yolo[0], 
                                 final_result_yolo[1], 
                                 final_result_yolo[2], 
                                 final_result_yolo[3]))
                brisque_score = round(brisque.score(image_), 2)
                
                reward = - 0.01*brisque_score
                
                done["__all__"] = True
                rew = {self.low_level_agent_id: reward}
                rew["high_level_agent"] = image_asses
                obs["high_level_agent"] = (obs_feat, hist, bins)
                
                im = Image.fromarray(image)
                filename = r'D:\hier' + "\img_%s_%s_%s.png" % (str(self.action_main),
                                                          str(action_lens), str(brisque_score))
                im.save(filename)

                
        else:
            rew = {self.low_level_agent_id: 0}
            done[self.low_level_agent_id] = True
            rew["high_level_agent"] = image_asses
            obs["high_level_agent"] = (obs_feat, hist, bins)
            
        im = Image.fromarray(image)
        filename = r'D:\hier' + "\img_%s_%s_%s_%s.png" % (str(self.action_main), str(image_asses),
                                                         str(action_lens), str(flag))
        im.save(filename)
        
        print('action:', action, 'info:',  done, rew)
        #self.writer.writerow([self.action_main, action_lens, flag])
        return obs, rew, done, {}
                    


In [6]:
from ray.tune.registry import register_env

def env_creator(env_config):
    return  BaslerEnv(env_config)

register_env("BaslerEnv", env_creator)

In [7]:
# Create an instant camera object with the camera device found first.
# camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
# camera.Open()

In [8]:
from ray.rllib.models import ModelCatalog
from ray import tune
from ray.tune import grid_search

In [9]:
# from ray.rllib.agents.impala.vtrace_tf_policy import VTraceTFPolicy
# from ray.rllib.agents.impala import ImpalaTrainer
# import ray.rllib.agents.impala as impala

# ray.init()

# def policy_mapping_fn(agent_id):
#     if agent_id.startswith("low_level_"):
#         return "low_level_policy"
#     else:
#         return "high_level_policy"
    
    
# stop = {"training_iteration": 2000,
#         "timesteps_total": 1000,
#         "episode_reward_mean": 0.0,}

# # policy, env, gamma=0.99, n_steps=128, ent_coef=0.01, 
# # learning_rate=0.00025, vf_coef=0.5, 
# # max_grad_norm=0.5, lam=0.95, 
# # nminibatches=4, noptepochs=4, 
# # cliprange=0.2, 
# # cliprange_vf=None, 
# # verbose=0, tensorboard_log=None, 
# # _init_setup_model=True, 
# # policy_kwargs=None, 
# # full_tensorboard_log=False, seed=None, n_cpu_tf_sess=None


# config = {
#     "env": BaslerEnv,
#     "num_workers": 1,
#     "entropy_coeff": 0.01,
#     #"rollout_fragment_length": 128,
#     "train_batch_size": 128,
#     "learner_queue_timeout": 1600,
#     #"sgd_minibatch_size": 128,
#     "lr": 0.00025,#tune.grid_search([0.01, 0.001, 0.0001]),
#     "multiagent": {
#         "policies": {
            
#             "high_level_policy": (VTraceTFPolicy, 
#                                   Box(0, 255, (2048,)),#(len(np.zeros((404, 417, 3)).flatten()),)),
#                                   Discrete(148), 
#                                   {"gamma": 0.99}),
            
#             "low_level_policy": (VTraceTFPolicy,
#                                  Box(0, 255, (2048,)),#(len(np.zeros((404, 417, 3)).flatten()),)),
#                                  MultiDiscrete([69, 99]), 
#                                  {"gamma": 0.0}),
            
# #             "low_level_policy": (PPOTFPolicy,
# #                                  Tuple([
# #                                      lens_env.observation_space,
# #                                      Discrete(148)
# #                                  ]), maze.action_space, {
# #                                      "gamma": 0.0
# #                                  }),
#         },
#         "policy_mapping_fn": function(policy_mapping_fn),
#     },
#     "framework": "tf",#"torch", #if args.torch else "tf",
#     # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
#     "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
# }

# #from ray.rllib.agents.trainer_template import build_trainer

# # <class 'ray.rllib.agents.trainer_template.MyCustomTrainer'>
# # MyTrainer = build_trainer(
# #     name="MyCustomTrainer",
# #     default_policy=MyTFPolicy)

# # ray.init()
# # tune.run(MyTrainer
# trainer = impala.ImpalaTrainer(env= "BaslerEnv", config=config)
# results = tune.run("PPO", config=config, stop=stop,  
#                    restore=r'C:\Users\CIG\Documents\MATLAB\ppo_pic\checkpoint-21')

In [10]:
#ModelCatalog.register_custom_model("PPO_2",PPO2)

In [11]:
#len(np.zeros((404, 417, 3)).flatten())

In [12]:
ray.init()

def policy_mapping_fn(agent_id):
    if agent_id.startswith("low_level_"):
        return "low_level_policy"
    else:
        return "high_level_policy"
    
    
stop = {"training_iteration": 2000,
        "timesteps_total": 1000,
        "episode_reward_mean": 0.0,}

# policy, env, gamma=0.99, n_steps=128, ent_coef=0.01, 
# learning_rate=0.00025, vf_coef=0.5, 
# max_grad_norm=0.5, lam=0.95, 
# nminibatches=4, noptepochs=4, 
# cliprange=0.2, 
# cliprange_vf=None, 
# verbose=0, tensorboard_log=None, 
# _init_setup_model=True, 
# policy_kwargs=None, 
# full_tensorboard_log=False, seed=None, n_cpu_tf_sess=None


config = {
    "env": BaslerEnv,
    "num_workers": 1,
    "entropy_coeff": 0.01,
    "rollout_fragment_length": 128,
    "train_batch_size": 128,
    "sgd_minibatch_size": 128,
    "lr": 0.00025,#tune.grid_search([0.01, 0.001, 0.0001]),
    "multiagent": {
        "policies": {
            
            "high_level_policy": (PPOTFPolicy, 
                                  Tuple((spaces.Box(0, 2, (2048, )),
                                               spaces.Box(0, np.inf, (10, )), 
                                               spaces.Box(0, np.inf, (11, ))
                                              )),#(len(np.zeros((404, 417, 3)).flatten()),)),
                                  Discrete(148), 
                                  {"gamma": 0.99}),
            
            "low_level_policy": (PPOTFPolicy,
                                 Box(0, 255, (2048,)),#(len(np.zeros((404, 417, 3)).flatten()),)),
                                 MultiDiscrete([69, 99]), 
                                 {"gamma": 0.0}),
            
#             "low_level_policy": (PPOTFPolicy,
#                                  Tuple([
#                                      lens_env.observation_space,
#                                      Discrete(148)
#                                  ]), maze.action_space, {
#                                      "gamma": 0.0
#                                  }),
        },
        "policy_mapping_fn": function(policy_mapping_fn),
    },
    "framework": "tf",#"torch", #if args.torch else "tf",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
}

#from ray.rllib.agents.trainer_template import build_trainer

# <class 'ray.rllib.agents.trainer_template.MyCustomTrainer'>
# MyTrainer = build_trainer(
#     name="MyCustomTrainer",
#     default_policy=MyTFPolicy)

# ray.init()
# tune.run(MyTrainer
trainer = ppo.PPOTrainer(env= "BaslerEnv", config=config)
# results = tune.run("PPO", config=config, stop=stop,  
#                    restore=r'C:\Users\CIG\Documents\MATLAB\ppo_pic\checkpoint-21')

2021-05-14 12:57:33,974	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-05-14 12:57:35,910	WARNING sample.py:404 -- DeprecationWarning: wrapping <function policy_mapping_fn at 0x00000212E1394AF8> with tune.function() is no longer needed
2021-05-14 12:57:35,954	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-05-14 12:57:35,954	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=34880) WARNING:tensorflow:From c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\tensorflow\python\compat\v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=34880) Instructions for updating:
(pid=34880) non-resource variables are not supported in the long term


(pid=34880) 417 404
(pid=34880) 148
(pid=34880) eCOMCaspErr: 0 0
(pid=34880) 417 404
(pid=34880) 148
(pid=34880) ---------------lens env init-------------------


(pid=34880) Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\CIG/.cache\torch\hub\master.zip
(pid=34880) Fusing layers... 
(pid=34880) Model Summary: 224 layers, 7266973 parameters, 0 gradients
(pid=34880) Adding autoShape... 
(pid=34880) YOLOv5  2021-5-14 torch 1.8.0+cpu CPU
(pid=34880) 
(pid=34880) 2021-05-14 12:57:45,361	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!


(pid=34880) True


(pid=34880) 2021-05-14 12:57:46,304	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-14 12:57:48,074	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-14 12:57:49,101	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-14 12:57:55,229	INFO trainable.py:103 -- Trainable.setup took 19.275 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-05-14 12:57:55,229	WARNING util.py:47 -- Install gputil for GPU system monitoring.


In [13]:
#trainer.restore(r'C:\Users\CIG\ray_results\PPO_noref2\checkpoint_48\checkpoint-48')

In [ ]:
for i in range(100):
    # Perform one iteration of training the policy with PPO\n",
    result = trainer.train()
    print(pretty_print(result))
    print('-------', i, '-------')
# #     print(evaluation_results)
    if i % 80 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)

(pid=34880) 2021-05-14 12:58:01,964	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


(pid=34880) ExposureTimeRaw: 185000
(pid=34880) action: [19 79] info: {'__all__': False, 'low_level_1': True} {'low_level_1': 0, 'high_level_agent': -1}
(pid=34880) ExposureTimeRaw: 290000


(pid=34880) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=34880)   self.image = skimage.color.rgb2gray(self.image)


(pid=34880) action: [ 7 13] info: {'__all__': False} {'low_level_2': -1}


(pid=34880) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=34880)   self.image = skimage.color.rgb2gray(self.image)


...

(pid=34880) action: [12 90] info: {'__all__': False} {'low_level_2': -1}


(pid=34880) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=34880)   self.image = skimage.color.rgb2gray(self.image)


(pid=34880) action: [48 42] info: {'__all__': True} {'low_level_2': -0.3403, 'high_level_agent': -1}
(pid=34880) ExposureTimeRaw: 70000


(pid=34880) c:\users\cig\anaconda3\envs\rlautofocus\lib\site-packages\imquality\brisque.py:45: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
(pid=34880)   self.image = skimage.color.rgb2gray(self.image)


(pid=34880) action: [48 63] info: {'__all__': True} {'low_level_1': -0.5063000000000001, 'high_level_agent': 1}
(pid=34880) ExposureTimeRaw: 185000
(pid=34880) action: [47 56] info: {'__all__': False, 'low_level_1': True} {'low_level_1': 0, 'high_level_agent': -1}
(pid=34880) ExposureTimeRaw: 195000
(pid=34880) action: [48 97] info: {'__all__': False, 'low_level_2': True} {'low_level_2': 0, 'high_level_agent': -1}
(pid=34880) ExposureTimeRaw: 195000
(pid=34880) action: [47  5] info: {'__all__': False, 'low_level_3': True} {'low_level_3': 0, 'high_level_agent': -1}
(pid=34880) ExposureTimeRaw: 400000


In [ ]:
# for i in range(100):
#     # Perform one iteration of training the policy with PPO\n",
#     result = trainer.train()
#     print(pretty_print(result))
#     print('-------', i, '-------')
# # #     print(evaluation_results)
#     if i % 80 == 0:
#         checkpoint = trainer.save()
#         print("checkpoint saved at", checkpoint)

In [ ]:
checkpoint = trainer.save()

In [ ]:
# stop = {
#     "training_iteration": 100,
#     "timesteps_total": 1000,
# }

# results = tune.run("PPO", stop=stop, config=config, checkpoint_freq=5, verbose=1)

In [ ]:
#tensorboard --logdir=~/ray_results